# **Tutorial on Record Selection**
## METIS Summer School Pavia 2022
--------------------------------------------------------------------------------


# Getting started
This will install the requirements for record selection, openquake, etc...
Only execute if necessary, otherwise skip to the directories definition



In [ ]:
%%capture
#pip uninstall --yes numba

In [ ]:
%%capture
# pip install --pre numba

In [ ]:
%%capture
# pip install openquake.engine

Next step is to identify the directories for the various inputs, coming from ***PSHA*** and ***disaggregation***, the ground motion ***DB*** metadata and the ***outputs***. 

It is important to keep only disaggregation for 1 location `M_R_eps file`, since record selection will be done just for 1 site

In [ ]:
# These are created within the machine 
out_dir= r"C:\Users\Pablo\Documents\PhD\Other\Zagreb\GMSelection\output" # directory for outputs
fig_dir = r"C:\Users\Pablo\Documents\PhD\Other\Zagreb\GMSelection\output\figures"  # directory where the figures are stored
output_dir = r"C:\Users\Pablo\Documents\PhD\Other\Zagreb\GMSelection\output\outputGM"  # directory where ground motion outputs are stored
# These will come from github
mat_dir = r"C:\Users\Pablo\Documents\PhD\Other\Zagreb\GMSelection\Data\DB_final_6.mat"  # mat file with the Database of records - metadata flatfile
path_dis = r"C:\Users\Pablo\Documents\PhD\Other\Zagreb\GMSelection\Data\PSHA_Output\Disaggregation"  # output from OQ
path_haz = r"C:\Users\Pablo\Documents\PhD\Other\Zagreb\GMSelection\Data\PSHA_Output\Hazard"  # output from OQ

# ESM Database
In order to select records, we require a database to pull them from, it is important that it contains relevant information on the causative parameters as well as the intensities of the event. Here, the mat file contains all the relevant information about the records, such as spectral quantities, Magnitude, distance to station, depth, Vs30, etc.



---



Loading the functions and packages and creating directories... 

**Be mindful to change to the directory where the other functions are!**

In [ ]:
import os
import os.path
from os import path
import math

if path.exists(out_dir) == False:
  os.mkdir(out_dir)
if path.exists(output_dir) == False:
  os.mkdir(output_dir)
if path.exists(fig_dir) == False:
  os.mkdir(fig_dir)
if path.exists(output_dir) == False:
  os.mkdir(output_dir)

os.chdir(r"D:\Documents\PhD\METIS\Summer_school\Summer_school\to share")
import numpy as np
from utils_bsc import CS
from time import time
from OQProc2 import disagg_MReps, hazard
import os
import pickle
from matplotlib import pyplot as plt
import warnings
from Saveutils import fxn, saveMat, plot_select_recs

# Conditional spectrum 
The conditional spectrum, as previously stated is a target based record selection procedure which accounts for the variability in the hazard. 
The steps mainly followed by this procedure which will be detailed through the next steps are:
  1. Definition of target periods, intensities, GM, limits, etc.
  2. Disaggregation of the hazard for a given period ***T*** and a given ***probability of exceedance*** (or RP).
  3. Definition of the target spectrum, mean and variance.
  4. Ground motion selection through sampling.
  5. Optimization.

## The algorithm

Script used to select the records based on the Sa(T*) value. It is based (with some modifications) on: https://github.com/volkanozsarac/EzGM.git;

Calculation of SSEs is from "A Computationally Efficient Ground-Motion Selection Algorithm for Matching a Target Response
    Spectrum Mean and Variance" Jayaram, 2011:  https://journals.sagepub.com/doi/pdf/10.1193/1.3608002;

Any GMPE available in the OQ library can be used. If we set pinfo=1 we will get information about GMPE (e.g. for which
    IM it was derived)
make sure that you use same GMPE in hazard calculation (in OQ) and for the selection

Main processes about CS are found in the `utils_bsc` script, we will see some parts of the code within here as we execute the commands...



# Main Inputs

Now we set the inputs: level of intensity, number of records to select,Vs30 and scaling limits, etc.

Additionally we define the inputs for the GMPE to be used for the target spectrum

In [ ]:
poes = [0.1, 0.05]  # probability of exceedance in 50 years
T_target = [0.075, 0.10, 0.15, 0.20, 0.25, 0.30, 0.40, 0.50, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.50, 1.6,
            1.7, 1.8, 1.9, 2.00, 3.00, 4.00]  # periods of interest for target spectrum

nIM = 2     # IML for calculation (out of the list of poes)
T_star = 0.2  # period of interest
Mbin = 1.0  # the values used in hazard calculation (OQ)
dbin = 10   # the values used in hazard calculation (OQ)
nGM = 40      # number of GM to select
Vs30_lims=[600,2500] # Vs30 limits in m/s
maxSF =  4.0  # max scaling factor
minSF = 0.2 # min scaling factor


# Input for the GMPE (depends on the GMPE)
rake = 180
Vs30 = 800
Fhw = 1


# Disaggregation analysis
In order to create a target spectrum, it is necessary to understand which events will have the most contribution to the hazard, this will allow us to get the target scenario"s causative parameters:
 Magnitude (***M***), Distance (***R***) and ***$\epsilon$***



In [ ]:

# Read the results of the disaggregation and get M, R and epsilon values
imls = hazard(poes, path_haz, output_dir=fig_dir,
              rlz="hazard_curve-mean", i_save=1, i_show=1)  # IMs at the predefined levels (poes)
meanLst, modeLst = disagg_MReps(Mbin, dbin,T_star, path_dis, output_dir=fig_dir, 
                                          n_rows=1, filename="Mag_Dist_Eps")  # mean and mode scenarios from disaggregation (M, R and epsilon) --- slow



# Creating the target
The next step is to create the target spectra using the GMPE with the values obtained from the disaggregation analysis at the intensity level (IML) of choice.

First we get the required values:

In [ ]:
All_CS = []
j=nIM-1
print("IM level:", str(nIM))
# Initialize the CS class
CS1 = CS(mat_dir, T_star=T_star, gmpe="BooreAtkinson2008", pInfo=1)
mag = meanLst[j][0]
rjb = meanLst[j][1]
eps = meanLst[j][2]  # not used here
im_T_star = imls[0][j]
print("Poe is:", 100*poes[j], "% in 50 years.", "Mean Magnitude and Rjb are:", np.round(mag, 2), np.round(rjb, 2))
print("IM at the level", str(nIM), "is:", np.round(im_T_star, 3))
SOF, W, dip, Ztor, Rrup, Rx, Ryo, Z1 = CS1.getUnknown_params(M=mag, Fhw=Fhw, rake=rake, Rjb=rjb, Vs30=Vs30)




---


Now, we create the target spectrum using the required data for the GMPE and values from disaggregation. 


This will allow us to create our mean conditional target spectrum with:



 $\mu_{lnSa(T_i)|lnSa(T*)} = \mu_{lnSa}(M,R,T_i)+\rho(T_i,T*)\epsilon(T*)\sigma_{lnSa}(T_i)$



 Where  $\mu_{lnSa}(M,R,T_i)$ is the unconditional mean spectrum, computed simply through the GMPE




Where the correlation between periods  $\rho(T_i,T_j)$ is computed following Baker and Jayaram, 2008 (https://journals.sagepub.com/doi/pdf/10.1193/1.2857544)


Finally, **for the conditional dispersion** at each period, the calculations are done with the following equations, also found in the script:

Where this dispersion is found in the diagonal of the conditional covariance matrix is found as:


$\Sigma_{cond} = \Sigma -\frac{\Sigma_{cross}\Sigma_{cross}^T} {\sigma_{lnSa}(M,R,T^*)^2}$

Where $\Sigma$ is the covariance matrix:

$\Sigma = \left[\begin{matrix}
\sigma^2_{T_i} & \sigma_{T_i,T_j}\\
\sigma_{T_j,T_i} & \sigma^2_{T_j}
\end{matrix}\right]$

And $\Sigma_{cross}$ is a matrix of covariance between the periods $T^*, T_i$ and $T_j$

$\Sigma_{cross} = \left[\begin{matrix}
\sigma_{T_i,T^*}\\
\sigma_{T^*,T_j}
\end{matrix}\right]$

Where the terms between periods $T_i$ and $T_j$ are computed as:

$\sigma_{T_i,T,j}=\rho(T_i,T_j)\sigma_{lnSa}\sigma_{lnSa}$



In [ ]:
# Create the target spectra
CS1.create(site_param={"vs30": Vs30},
                                rup_param={"rake": rake, "mag": [mag]},
                                dist_param={"rjb": [rjb], "rrup": Rrup, "rx": Rx, "ry0": Ryo}, Hcont=None, T_Tgt_range=T_target, im_T_star=im_T_star,
                                epsilon=None, cond=1, useVar=1, corr_func="baker_jayaram")
print("Target distribution is defined")

CS1.plot_Target(j, save=0, show=1, name_dir=fig_dir)


# Ground motion selection
This next step is where the simulations are performed, obtaining different spectral values for different records.

(See: https://journals.sagepub.com/doi/pdf/10.1193/1.3608002)

The first step is to probabilistically simulate a set of response spectra that comply with the target"s mean and variance through Monte Carlo sampling;


Then, for each simulated response spectrum, a ground motion with a similar response spectrum is then selected, and given that the Monte Carlo simulations have the target"s mean and variance, it is expected that the selected GM will have it as well.

### Greedy optimization

These records are then evaluated and optimized in terms of $\mu$lnSa and $\sigma$lnSa through the greedy optimization process.
The error is computed through: 


\begin{align}
\text{SSE } & = \sum_{j=1}^p\left[ \left( \hat{m}_{lnSa(T_j)}-\mu^{(t)}_{lnSa(T_j)} \right)^2+w\left( \hat{s}_{lnSa(T_j)}-\sigma^{(t)}_{lnSa(T_j)} \right)^2\right]. \\[1em]
\
\end{align}

Where $\hat{m}_{lnSa(T_j)}$ and $\hat{s}_{lnSa(T_j)}$ are the sample"s mean and standard deviation, and $\mu_{lnSa(T_j)}$ and $\sigma_{lnSa(T_j)}$ are the target"s mean and standard deviation, respectively.

In the end the records producing the lowest SSE are chosen for the final selection.

In [ ]:
# Select ground motion records
CS1.select(nGM=nGM, selection=2, Sa_def="RotD50", isScaled=1, maxScale=maxSF, minScale=minSF, Mw_lim=[4, 8],
            DB_lim=["Ridge","NGA","GNS"], Vs30_lim=Vs30_lims, Rjb_lim=None, fault_lim=None, freefield=True, nTrials=20, weights=[1, 2, 0.3], seedValue=0, nLoop=20, penalty=1, tol=10)
CS1.plot_sel((j), save=1, show=1, initial=1, name_dir=fig_dir)  
All_CS.append(CS1)


print("Records selected!")

# Exporting data
Now that the CS algorithm is done, we have obtained our record set, this set contains information about the records, such as the event name, network and station (can be downloaded from the ESM database - https://esm-db.eu/#/home) or alternatively, the file name (if all the database is previously downloaded), scaling factors, ground motion data and causative parameters.

In [ ]:
from sys import intern
resdata={"Network":All_CS[0].eq_ID,"Station":All_CS[0].eq_ID,"Event":All_CS[0].eq_ID,"SF":All_CS[0].rec_scale,"dt":All_CS[0].dt,"npts":All_CS[0].nstp,"Mw":All_CS[0].rec_Mw,"R":All_CS[0].rec_Rjb,"Vs30":All_CS[0].rec_Vs30,"Fname1":All_CS[0].rec_h1,"Fname2":All_CS[0].rec_h2,"url":All_CS[0].eq_ID}
import pandas as pd 
ResDF = pd.DataFrame(data=resdata)

# get event names, network and station for ESM search
for inn in range(nGM):
  s = ResDF.Fname1[inn]
  start = s.find(".D.") + len(".D.")
  end = s.find(".ACC.")
  EvenID = s[start:end]
  partitioned_string = s.partition(".")
  ResDF.Network[inn]=partitioned_string[0]
  ResDF.Event[inn]=EvenID
  start = s.find(partitioned_string[0]) + len(partitioned_string[0])+1
  end1 = s.find("..")
  end2 =s.find(".00.")
  end = max(end1,end2)
  Statr = s[start:end]
  ResDF.Station[inn]=Statr
  try:
    urli="https://esm-db.eu/#/waveform/"+ResDF.Network[inn]+"/"+ResDF.Station[inn]+"/00/"+EvenID+"/HG"
  except:
    urli="error"
  ResDF.url[inn]=urli
# export table
ResDF.to_csv(output_dir+"/RecSelection_IM_"+str(nIM)+".csv")
ResDF

# Selected records

Now with everything done, the records can be processed in many ways and finally downloaded for time history analysis...

In [ ]:
for k in range(nGM):
  plt.loglog(All_CS[0].T,np.exp(All_CS[0].rec_spec[k,:]),"silver")
#plt.loglog(All_CS[0].T,np.exp(All_CS[0].mu_ln),"r"); # Target
plt.xlabel("Period [s]",fontsize=14)
plt.ylabel("Spectral acceleration [g]",fontsize=14)
plt.xlim(All_CS[0].T[0],All_CS[0].T[-1])

See the selected records, relevant data and download at the ESM website.

Ideally, it is better to have the full DB files so that they can be selected, processed and used automatically for the relevant analyses

In [ ]:
for zz in ResDF.url:
  print(zz)

More information on CS can be found [here](https://www.youtube.com/watch?v=1B6d9rrMPu0) (video by Jack Baker on CMS)

If you are more familiar with MATLAB, we have developed record selection scripts based on the ones by Jack Baker (https://github.com/bakerjw/CS_Selection) originally developed for NGA, modified for the ESM database.